In [1]:
from llama_index.legacy import SimpleDirectoryReader, StorageContext, ServiceContext
from llama_index.legacy.indices.vector_store import VectorStoreIndex
from llama_iris import IRISVectorStore

In [2]:
import getpass
import os
from dotenv import load_dotenv

load_dotenv(override=True)

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [3]:
documents = SimpleDirectoryReader("../data/patient").load_data()
print("Document ID:", documents[0].doc_id)

Document ID: 858d4edb-42d9-438c-8e96-37c633c739ea


In [3]:
username = 'demo'
password = 'demo' 
hostname = os.getenv('IRIS_HOSTNAME', 'localhost')
port = '1972' 
namespace = 'USER'
CONNECTION_STRING = f"iris://{username}:{password}@{hostname}:{port}/{namespace}"

In [4]:
vector_store = IRISVectorStore.from_params(
    connection_string=CONNECTION_STRING,
    table_name="patient_report",
    embed_dim=1536,  # openai embedding dimension
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
# service_context = ServiceContext.from_defaults(
#     embed_model=embed_model, llm=None
# )

In [5]:
#reconnect vector store
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
query_engine = index.as_query_engine()

In [10]:
# This creates a persistent vector store (as a SQL table) under the hood and stores the document embeddings. This should only be called once!
index = VectorStoreIndex.from_documents(
    documents, 
    storage_context=storage_context, 
    show_progress=True, 
    # service_context=service_context,
)
query_engine = index.as_query_engine()

g:\healthhack2025\hackathon-2024\iris-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 2/2 [00:03<00:00,  1.83s/it]


In [ ]:
# To add documents to an existing vector store:
storage_context = StorageContext.from_defaults(vector_store=vector_store)
for doc in documents:
    index.insert(document=doc, storage_context=storage_context)

Parsing nodes: 100%|██████████| 1/1 [00:00<00:00, 996.27it/s]


In [7]:
#patient dietary assessment
import textwrap
response = query_engine.query("what are the patient dietary concerns?")
print(textwrap.fill(str(response), 100))

The patient dietary concerns include high sodium and refined carbohydrate intake, excessive
consumption of processed foods, and a low intake of fresh vegetables and fruits. These dietary
patterns are likely contributing to suboptimal glycemic control in the patient.


In [8]:
#patient medical history
import textwrap
response = query_engine.query("what are the patient medical history concerns?")
print(textwrap.fill(str(response), 100))

The patient's medical history includes hypertension, type 2 diabetes mellitus, and hyperlipidemia.
